In [ ]:
from langchain_community.llms import Ollama
# Initialize LLM
llm = Ollama(
    model="gemma3:1b",
    temperature=0.3,
    num_ctx=2048  # Context window size
)
# Test LLM
print(llm.invoke("Explain quantum computing in simple terms"))

In [10]:
from langchain_community.embeddings import OllamaEmbeddings
# Initialize compatible embeddings
embedder = OllamaEmbeddings(
    model="nomic-embed-text",  # Same model for consistency
    num_ctx=512  # Smaller context for embeddings
)
# Test embeddings
embeddings = embedder.embed_query("Test sentence")
print(f"Embedding dimensions: {len(embeddings)}")

Embedding dimensions: 768


In [ ]:
# Load CSV data
from langchain_community.document_loaders import CSVLoader
import os
# Load CSV — set Issue column as the main content
loader = CSVLoader(
    file_path=os.path.abspath("tickets.csv"),
    source_column="Issue",  # Column with user queries
    metadata_columns=["Resolution"],  # Preserve these as metadata
    encoding="utf-8-sig" # Handle BOM in UTF-8 files
)

documents = loader.load()

In [16]:
# RAG pipeline remains the same
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Define the prompt template
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Use the provided context to answer the question.\n"
    "If no solution exists, respond with: ""Please create a support ticket for this issue."
    "Context: {context}\n"
    "Question: {question}\n"
    "Answer:"    
)

# Create the vector store with cosine distance strategy
vectorstore = FAISS.from_documents(
    documents,
    embedder,
    #distance_strategy="COSINE"
)

# Create retriever from the vector store
retriever = vectorstore.as_retriever()

# Create the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
# Run Query
question = "Run solve having error"
raw_response = rag_chain.invoke(question)
raw_response

'Please create a support ticket for this issue.'